<a href="https://colab.research.google.com/github/oskart20/MNIST_ml/blob/master/MNIST_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Clean the current env
#import os, signal
#os.kill(os.getpid(), signal.SIGKILL)

#Create input folder and download dataset
!rm -rf input
!mkdir -p input
!cd input/
!pip install gdown
!gdown https://drive.google.com/uc?id=1DOW6s1DpGUOsTJssnbjrM6MM27KT6s4K 

!unzip MNIST.zip -d input/

In [0]:
#Import some packages to use
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

#To see our directory
import os
import random
import gc   #Garbage collector for cleaning deleted data from memory

In [0]:
train_dir = '/content/input/MNIST/MNIST_JPG_training'
test_dir = '/content/input/MNIST/MNIST_JPG_testing'

train_digits = []
for j in range(10):
  temp = ['/content/input/MNIST/MNIST_JPG_training/{}/{}'.format(j, i) for i in os.listdir(train_dir+'/{}'.format(j))]
  train_digits.extend(temp[:5400]) #slice the dataset and use 5400 in each class

test_digits = []
for j in range(10):
  temp = ['/content/input/MNIST/MNIST_JPG_testing/{}/{}'.format(j, i) for i in os.listdir(test_dir+'/{}'.format(j))]
  test_digits.extend(temp[:800]) #slice the dataset and use 800 in each class

# shuffle it randomly
random.shuffle(train_digits)
random.shuffle(test_digits)

#Clear lists that are useless
del temp
gc.collect()   #collect garbage to save memory

In [0]:
#Lets declare our image dimensions
#we are using coloured images. 
nrows = 28
ncolumns = 28
channels = 3

#A function to read and process the images to an acceptable format for our model
def read_and_process_image(list_of_images, training=True):
    """
    Returns two arrays: 
        X is an array of resized images
        y is an array of labels
    """
    X = [] # images
    y = [] # labels

    for image in list_of_images:
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        img = img[...,::-1]
        X.append(img)  #Read the image
        #get the labels
        if training==True:
          y.append(image[-5:-4])    
    return X, y

In [0]:
#get the train and label data
X, y = read_and_process_image(train_digits)

In [0]:
#Lets view some of the pics
plt.figure(figsize=(20,10))
columns = 5
for i in range(columns):
    plt.subplot(5 / columns + 1, columns, i + 1)
    plt.imshow(X[i])

In [0]:
import seaborn as sns
del train_digits
gc.collect()

#Convert list to numpy array
X = np.array(X)
y = np.array(y)

#Lets plot the labels to be sure we just have 10 classes
sns.countplot(y)
plt.title('Labels for 0 through 9')

In [0]:
print("Shape of train images is:", X.shape)
print("Shape of labels is:", y.shape)

In [0]:
#Lets split the data into train and validation set
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2)

from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)


print("Shape of train images is:", X_train.shape)
print("Shape of validation images is:", X_val.shape)
print("Shape of labels is:", y_train.shape)
print("Shape of labels is:", y_val.shape)

In [0]:
#clear memory
del X
del y
gc.collect()

#get the length of the train and validation data
ntrain = len(X_train)
nval = len(X_val)

In [0]:
#We will use a batch size of 32. Note: batch size should be a factor of 2.***4,8,16,32,64...***
batch_size = 32

from keras.layers import *
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import backend as K
K.set_image_dim_ordering('th')

model = models.Sequential()
# we use relu as activation function as relu is appropriate for this task and most used in similiar cases
activation_func = 'relu'

model.add(Conv2D(64, (20, 20), activation=activation_func, padding="same", data_format='channels_last', input_shape=(28, 28, 3)))
model.add(MaxPooling2D((2, 2), padding="same", data_format='channels_last'))
model.add(Conv2D(32, (16, 16), activation=activation_func, padding="same", data_format='channels_last'))
model.add(MaxPooling2D((2, 2), padding="same", data_format='channels_last'))
model.add(Conv2D(32, (12, 12), activation=activation_func, padding="same", data_format='channels_last'))
model.add(MaxPooling2D((2, 2), padding="same", data_format='channels_last'))
model.add(Conv2D(128, (6, 6), activation=activation_func, padding="same", data_format='channels_last'))
model.add(MaxPooling2D((2, 2), padding="same", data_format='channels_last'))
model.add(Conv2D(128, (3, 3), activation=activation_func, padding="same", data_format='channels_last'))
model.add(MaxPooling2D((2, 2), padding="same", data_format='channels_last'))
model.add(Flatten())
model.add(Dropout(0.2))  #Dropout for regularization
model.add(Dense(2048, activation=activation_func))
model.add(Dense(2048, activation=activation_func))
model.add(Dropout(0.1))
model.add(Dense(768, activation=activation_func))
model.add(Dense(256, activation=activation_func))
model.add(Dense(10, activation='softmax'))


In [0]:
#Lets see our model
model.summary()

In [0]:
#We'll use the RMSprop optimizer with a learning rate of 0.0001
#We'll use categorical_crossentropy loss because its a categorical classification
model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [0]:
#Lets create the augmentation configuration
#This helps prevent overfitting
train_datagen = ImageDataGenerator(rescale=1./255,   #Scale the image between 0 and 1
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=False,)

val_datagen = ImageDataGenerator(rescale=1./255)  #We do not augment validation data. we only perform rescale

In [0]:
#Create the image generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

In [0]:
#The training part
#We train for 10 or more epochs with about 1350 steps per epoch
history = model.fit_generator(train_generator,
                              steps_per_epoch=ntrain // batch_size,
                              epochs=10,
                              validation_data=val_generator,
                              validation_steps=nval // batch_size)

In [0]:
#lets plot the train and val curve
#get the details form the history object
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# last_acc = history.history['val_acc'][len(history.history['val_acc'])-1]

#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

In [0]:
#Now lets predict on the first 20 Images of the test set
X_test, y_test = read_and_process_image(test_digits)
X_test = np.array(X_test)
y_test = np.array(y_test)

y_test = to_categorical(y_test)

test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:

i = 0
text_labels = []
plt.figure(figsize=(50,30))
# display 10 images from the test batch and the predicted labels with according percentage 
for batch in test_datagen.flow(X_test, batch_size=1):
    pred = model.predict(batch)
    pred = pred.tolist()
    data = pred[0]
    prediction = max(data)
    text_labels.append('{}:{}'.format(data.index(prediction), prediction*100))
    plt.subplot(5 / columns + 1, columns, i + 1)
    plt.title(text_labels[i])
    imgplot = plt.imshow(batch[0])
    i += 1
    if i % 10 == 0:
        break
plt.show()
scores = model.evaluate(X_test, y_test, verbose=0)
acc_print = '%.2f%%' % (scores[1]*100)
print('Accuracy: %.2f%%' % (scores[1]*100))
print('Loss: %.2f%%' % (scores[0]))
print('Error rate: %.2f%%' % ((1-scores[1])*100))

In [0]:
# save model
model.save('model_MNIST_{}_{}_Oskar-Thaeter_v1.h5'.format(activation_func, acc_print))